In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/RestaurantReviewProject/Restaurant_Reviews.tsv',delimiter="\t",quoting=3)

dataset.tail()
print(type(dataset))
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>


,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [2]:
import pandas as pd



In [4]:
print("no of reviews = ",dataset.shape)

no of reviews =  (1000, 2)


In [5]:
import re
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Preprocessing

### Stop Words
Stop words are common words like ‘the’, ‘and’, ‘I’, etc. that are very frequent in text, and so don’t convey insights into the specific topic of a document. We can remove these stop words from the text in a given corpus to clean up the data, and identify words that are more rare and potentially more relevant to what we’re interested in.  
  
By default, NLTK (Natural Language Toolkit) includes a list of 40 stop words, including: “a”, “an”, “the”, “of”, “in”, etc.  
  
Now we need to remove those unnessart stopwords to clean our data (normalizing the data)

### NOTE
the default stopwords from the nltk contains "not" word but it is very impartant in this project  
so we have to remove it from the stopWords


In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# print(review)
# print(dataset['Review'][0])
# review = re.sub('[^a-zA-Z]',' ',dataset['Review'][0])
# print(review)
stopwords_list = stopwords.words('english')
print(stopwords_list)



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Corpus
A corpus is a collection of authentic text or audio organized into datasets. Authentic here means text written or audio spoken by a native of the language or dialect.



### PorterStemming
The idea of stemming is a sort of normalizing method. Many variations of words carry the same meaning, when tense is not involved.

#### What does the Porter Stemmer do? 
The Porter stemming algorithm (or 'Porter stemmer') is a process for removing the commoner morphological and inflexional endings from words in English. Its main use is as part of a term normalisation process that is usually done when setting up Information Retrieval systems.  
#### EXAMPLE
[pythoner, pythonista, pythoning, pythonly, pythoned..] ==> python



In [12]:

corpus = []
ps = PorterStemmer()
stopWords = stopwords.words('english')
stopWords.remove('not')

for i in range(1000):
  review = re.sub('[^a-zA-Z]',' ',dataset['Review'][i])
  review = review.lower()
  review = review.split()
  
  reviewStemmed = []
  for word in review:
    if not(word in set(stopWords)):
      reviewStemmed.append(ps.stem(word))
  review = ' '.join(reviewStemmed)
  corpus.append(review)
  




In [ ]:
corpus


In [10]:
from sklearn.feature_extraction.text import CountVectorizer


## CountVectorizer
  Countvectorizer is a method to convert text to numerical data.  
  text = [‘Hello my name is james, this is my python notebook’]  
The text is transformed to a sparse matrix as shown below. 
![](https://miro.medium.com/max/909/1*64-SAdEtdZW2Et8HZzqJYA.png)


We have 8 unique words in the text and hence 8 different columns each representing a unique word in the matrix. The row represents the word count. Since the words ‘is’ and ‘my’ were repeated twice we have the count for those particular words as 2 and 1 for the rest.    
  For more refer:  https://towardsdatascience.com/basics-of-countvectorizer-e26677900f9c



In [19]:
cv = CountVectorizer(max_features=2000)
X= cv.fit_transform(corpus).toarray()


y_arr = dataset.iloc[:,-1].values


In [21]:
print(len(X[0]))

1566


In [22]:
y = pd.DataFrame(data = y_arr,columns=["Liked"])

In [ ]:
y.head()

,Liked
0,1
1,0
2,0
3,1
4,1


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
## Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:
print(cm)

[[48 48]
 [18 86]]


In [27]:
## accuracy score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print("ACCURACY =",accuracy)

ACCURACY = 0.67
